<a href="https://colab.research.google.com/github/gylam/siads696-sprsum2024-team05/blob/main/RW_Theme_Detection_data_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import os

In [ ]:
url = "https://api.reliefweb.int/v1/reports?appname=apidoc&query[fields][]=language&query[value]=English&profile=full&limit=1"
response = requests.get(url)
rw_dict = response.json()
rw_dict['data'][0]['fields'].keys()


dict_keys(['id', 'title', 'status', 'body', 'origin', 'primary_country', 'country', 'source', 'language', 'theme', 'format', 'url', 'url_alias', 'body-html', 'date'])

### Retrieve ReliefWeb Reports - to return the required fields
['theme_id','theme_name', 'report_id', 'title', 'text', 'url]

In [ ]:
def un_theme_data_retrievel_df(baseurl, limit, offset):

  '''
  Calls the ReliefWeb API to return the following:
  'theme_id','theme_name', 'report_id', 'title', 'text', 'url

  Returns df
  '''

  #keeping count of number of reports added to final df
  report_number = 0

  #initializing df
  df = pd.DataFrame(columns = ['theme_id', 'theme_name', 'report_id', 'text'])

  url = f'{baseurl}{limit}&offset={offset}'

  number_of_API_calls = 1
  while report_number < number_of_API_calls * int(limit):

    response = requests.get(url)
    rw_dict = response.json()

    all_reports_data=[]
    for i in np.arange(int(limit)):
      all_reports_data.append(rw_dict['data'][i]['fields'])


    for data in all_reports_data:
      try:
      # #some reports don't have a theme
      # if 'theme' not in data.keys() and 'body' not in data.keys() and 'id' not in data.keys():
      #   print('No Theme')
      # else:
        #create theme df
        theme_id = []
        theme_name = []

        for i in data['theme']:
          theme_id.append(list(i.values())[0])
          theme_name.append(list(i.values())[1])

        df_theme = pd.DataFrame(list(zip(theme_id, theme_name)), columns=['theme_id','theme_name'])

        #add report_id to df_theme to merge df_text on later
        id_col = []
        for i in np.arange(len(df_theme)):
          id_col.append(data['id'])
        df_theme['report_id'] = id_col

        #create report text df
        report_id = [data['id']]
        text = [data['body']]
        title = [data['title']]
        current_url = [list(rw_dict['links']['next'].values())[0]]
        latitude = [data['primary_country']['location']['lat']]
        longitude = [data['primary_country']['location']['lon']]
        country_iso3 = [data['primary_country']['iso3']]
        country_name = [data['primary_country']['name']]
        date_created = [data['date']['created']]
        source_id = [data['source'][0]['id']]
        source_name = [data['source'][0]['name']]
        format = [data['format'][0]['name']]
        df_text = pd.DataFrame(list(zip(report_id, title, text, [data['url']], latitude, longitude, country_iso3,
                                        country_name, date_created, source_id, source_name, format )),
                               columns = ['report_id', 'title', 'text', 'url', 'latitude', 'longitude', 'country_iso3',
                                        'country_name', 'date_created', 'source_id', 'source_name', 'format'])
        #merge theme and text dfs
        df_merged = df_theme.merge(df_text, how = 'right', on= 'report_id' )

        #update df
        df= pd.concat([df, df_merged])


        #update report count
        report_number += 1

        #update url for next report API call
        url= list(rw_dict['links']['next'].values())[0]

      except:
        #update url for next API call
        url= list(rw_dict['links']['next'].values())[0]


  print(f"number of reports = {report_number}")
  print(url)

  return df.reset_index().drop('index', axis = 1)



In [ ]:
##testing the retrieval function

baseurl = "https://api.reliefweb.int/v1/reports?appname=apidoc&query[fields][]=language&query[value]=English&profile=full&limit="
un_theme_data_retrievel_df(baseurl, limit = 1, offset = 0)#['theme_name'])

number of reports = 1
https://api.reliefweb.int/v1/reports?appname=apidoc&query%5Bfields%5D%5B0%5D=language&query%5Bvalue%5D=English&profile=full&limit=1&offset=1


,theme_id,theme_name,report_id,text,title,url,latitude,longitude,country_iso3,country_name,date_created,source_id,source_name,format
0,4587,Agriculture,3901366,SG/SM/21556\n\nThe following statement was iss...,"Secretary-General Urges Renewal, Full Implemen...",https://reliefweb.int/node/3901366,49.32,31.32,ukr,Ukraine,2022-10-29T02:49:39+00:00,1470.0,UN Secretary-General,News and Press Release
1,4593,Food and Nutrition,3901366,SG/SM/21556\n\nThe following statement was iss...,"Secretary-General Urges Renewal, Full Implemen...",https://reliefweb.int/node/3901366,49.32,31.32,ukr,Ukraine,2022-10-29T02:49:39+00:00,1470.0,UN Secretary-General,News and Press Release


### Pickle and Store data retrieved from API

In [ ]:
#Call the data retrieval function
#create pickle files from RW API text and 'theme' data
#use the offset parameter to keep adding reports

baseurl = "https://api.reliefweb.int/v1/reports?appname=apidoc&query[fields][]=language&query[value]=English&profile=full&limit="

limit = str(400)
offset = str(0)#add 1600 to offset to grab next reports/calls

for i in np.arange(4): #makes 1600 reports/calls, 400 reports at a time
  df = un_theme_data_retrievel_df(baseurl, limit, offset)
#   # df.to_pickle('test_untheme_pickle'+str(i))
  df.to_pickle('/content/drive/MyDrive/Milestone II/Milestone_II_project/data/RW_UN_themes_data_pickle'+offset)
  offset = str(int(offset) + int(limit))

print(offset)

# #takes about 45 secs to retrieve 4000 reports

number of reports = 599
https://api.reliefweb.int/v1/reports?appname=apidoc&query%5Bfields%5D%5B0%5D=language&query%5Bvalue%5D=English&profile=full&limit=400&offset=800
number of reports = 590
https://api.reliefweb.int/v1/reports?appname=apidoc&query%5Bfields%5D%5B0%5D=language&query%5Bvalue%5D=English&profile=full&limit=400&offset=1200
number of reports = 633
https://api.reliefweb.int/v1/reports?appname=apidoc&query%5Bfields%5D%5B0%5D=language&query%5Bvalue%5D=English&profile=full&limit=400&offset=1600
number of reports = 608
https://api.reliefweb.int/v1/reports?appname=apidoc&query%5Bfields%5D%5B0%5D=language&query%5Bvalue%5D=English&profile=full&limit=400&offset=2000
1600


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Read in the stored pickle files

In [ ]:
df_0 = pd.read_pickle("/content/drive/MyDrive/Milestone II/Milestone_II_project/data/RW_UN_themes_data_pickle0")
print(f'The fields returned by the above API call include: \n \n{list(df_0.columns)}')


The fields returned by the above API call include: 
 
['theme_id', 'theme_name', 'report_id', 'text', 'title', 'url', 'latitude', 'longitude', 'country_iso3', 'country_name', 'date_created', 'source_id', 'source_name', 'format']


In [ ]:

# Get the list of all pickle files in directory
path = "/content/drive/MyDrive/Milestone II/Milestone_II_project/data/"
dir_list = os.listdir(path)
# prints all files
print(f'''Files and directories in {path} :\n{dir_list}\n Total number of file = {len(dir_list)}''')


Files and directories in /content/drive/MyDrive/Milestone II/Milestone_II_project/data/ :
['RW_UN_themes_data_pickle0', 'RW_UN_themes_data_pickle400', 'RW_UN_themes_data_pickle800', 'RW_UN_themes_data_pickle1200']
 Total number of file = 4


#### create a single df from all pickle files in directory

In [ ]:
#Read in all pickle files in a single df
base_path = '/content/drive/MyDrive/Milestone II/Milestone_II_project/data/'


def read_pickles(base_path):
  dir_list = os.listdir(base_path)
  df = pd.DataFrame(columns =['report_id', 'title', 'text', 'url', 'latitude', 'longitude', 'country_iso3',
                                        'country_name', 'date_created', 'source_id', 'source_name', 'format'])

  for filename in dir_list:
    path = f'{base_path}/{filename}'
    #read content pickle content into df
    df_new = pd.read_pickle(path)
    #update df
    df= pd.concat([df, df_new])
  return df

df = read_pickles(base_path)

In [ ]:
#checking df shape and number of reports represented
df_shape = df.shape
print(f'Shape of the Dataframe (multiple rows per unique report) = {df_shape[0]} rows and {df_shape[1]} columns')
report_number = len(df['report_id'].unique())

print(f'Number of unique ReliefWeb reports retrieved = {report_number}')

Shape of the Dataframe (multiple rows per unique report) = 7388 rows and 14 columns
Number of unique ReliefWeb reports retrieved = 1975


#### Theme counts

In [ ]:
#Count of unique themes
# import counter class from collections module
from collections import Counter

#Creating a Counter class object using list as an iterable data container
theme_names = df['theme_name']

theme_counts = Counter(theme_names)

#printing counts
print(f'There are {len(theme_counts.keys())} Themes represented in this dataset.\nThey are: \n{list((theme_counts.keys()))}')
print(f'\n')
print(f'Counts of each unique "theme" in this dataset:\n{dict(theme_counts)}')


There are 20 Themes represented in this dataset.
They are: 
['Education', 'Food and Nutrition', 'Health', 'Protection and Human Rights', 'Shelter and Non-Food Items', 'Water Sanitation Hygiene', 'Disaster Management', 'Mine Action', 'Peacekeeping and Peacebuilding', 'Safety and Security', 'Coordination', 'Contributions', 'Climate Change and Environment', 'Logistics and Telecommunications', 'Gender', 'Agriculture', 'Recovery and Reconstruction', 'HIV/Aids', 'Humanitarian Financing', 'Camp Coordination and Camp Management']


Counts of each unique "theme" in this dataset:
{'Education': 692, 'Food and Nutrition': 766, 'Health': 1061, 'Protection and Human Rights': 1660, 'Shelter and Non-Food Items': 354, 'Water Sanitation Hygiene': 729, 'Disaster Management': 45, 'Mine Action': 98, 'Peacekeeping and Peacebuilding': 437, 'Safety and Security': 91, 'Coordination': 341, 'Contributions': 446, 'Climate Change and Environment': 32, 'Logistics and Telecommunications': 173, 'Gender': 45, 'Agricul

The dataset is not balanced. The most common theme is 'Gender' and least common is 'Camp Coordination and Camp Management'

#### Balance the dataset

In [ ]:
theme_list = list(theme_counts.keys())
row_count_per_theme = 2000

def balance_dataset(df, row_count_per_theme):

  df_tiny = pd.DataFrame(columns = df.columns)
  for theme in theme_list:
    df_new = df[df['theme_name'] == theme][:row_count_per_theme]
    # update df_tiny
    df_tiny= pd.concat([df_tiny, df_new])
  return df_tiny

balanced_df = balance_dataset(df, row_count_per_theme)
balanced_df.shape

(7388, 14)